In [1]:
import numpy as np
import torch
from transformers import LlamaTokenizer, AutoModelForCausalLM, AutoTokenizer

#model_id = "stabilityai/japanese-stablelm-instruct-alpha-7b"    
#model_id = "stabilityai/japanese-stablelm-base-alpha-7b"
model_id = "stabilityai/japanese-stablelm-base-gamma-7b"

#tokenizer = LlamaTokenizer.from_pretrained("novelai/nerdstash-tokenizer-v1", additional_special_tokens=['▁▁'])
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
)
#model.half()
model.eval()

def build_prompt(user_query, inputs="", sep="\n\n### "):
    sys_msg = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    p = sys_msg
    roles = ["指示", "応答"]
    msgs = [": \n" + user_query, ": "]
    if inputs:
        roles.insert(1, "入力")
        msgs.insert(1, ": \n" + inputs)
    for role, msg in zip(roles, msgs):
        p += sep + role + msg
    return p


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:

# this is for reproducibility.
# feel free to change to get different result
seed = 42
torch.manual_seed(seed)

# Infer with prompt without any additional input
user_inputs = {
    "user_query": "VR とはどのようなものですか？",
    "inputs": ""
}
prompt = build_prompt(**user_inputs)

tokens = tokenizer(
    prompt, 
    add_special_tokens=False, 
    return_tensors="pt"
)

pos = tokens.input_ids.shape[-1]

In [9]:
inputs = {
    'input_ids'           : tokens.input_ids,
    'attention_mask'      : tokens.attention_mask,
    'position_ids'        : torch.tensor([n for n in range(pos)], dtype=torch.int32),
    'use_cache'           : True,
    'return_dict'         : True
}

In [10]:
num_generate_tokens = 20

for nn in range(num_generate_tokens):

    res = model(**inputs)

    logits = res['logits'][0,-1,:].to('cpu').detach().numpy().copy()
    predicted_id = np.argmax(logits)
    if predicted_id == tokenizer.eos_token_id:
        print('** EOS token is generated.')
        break

    token = tokenizer.decode([predicted_id], skip_special_tokens=False)
    print(token, end='', flush=True)

    inputs = {
        'input_ids'           : torch.tensor([[predicted_id]], dtype=torch.int64),
        'attention_mask'      : torch.tensor([[1]], dtype=torch.int64),
        'position_ids'        : torch.tensor([[pos]], dtype=torch.int32),
        'past_key_values'     : res['past_key_values'],
        'use_cache'           : True,
        'return_dict'         : True
    }
    pos += 1

# past_key_values = ( ( [1,8,seq,128], [1,8,seq,128] ) *32 ) 


VRは、���想現実の略です。VRは、